In [ ]:
import pandas as pd
from math import cos, radians, sqrt
from sklearn.neighbors import BallTree
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point
import numpy as np
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px
import warnings
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:,.2f}'.format)
np.set_printoptions(suppress = True)
np.set_printoptions(precision = 2)

In [ ]:
raw = "C:\\Users\\taavi\\Desktop\\BPHIL\\Raw data\\"
clean = "C:\\Users\\taavi\\Desktop\\BPHIL\\Clean data\\"

In [ ]:
parcels = pd.read_csv(clean + 'blight_and_values.csv')
parcels = parcels.drop_duplicates(subset = 'parcelID')
parcels['blight'] = parcels['blight'].astype(str)
demos = pd.read_csv(clean + 'demos_values_blight.csv')
viols = pd.read_csv(clean + 'clean_viols.csv')

In [ ]:
color_continuous_scale=[
        '#440154', '#443983', '#31688e', '#21918c', '#35b779', '#90d743', '#fde725'
]

### 100 individul parcels

In [ ]:
fig = px.scatter_mapbox(parcels.sample(100), lat = 'lat', lon = 'lng', zoom = 10,
                        hover_data = ['nbrhd'])
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

### All parcels, uncolored

In [ ]:
fig = px.scatter_mapbox(parcels, lat = 'lat', lon = 'lng', zoom = 10,
                        hover_data = ['nbrhd'])
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

### Continuous viols_sqrt

In [ ]:
fig = px.scatter_mapbox(parcels, lat = 'lat', lon = 'lng', color = 'viols_sqrt', color_continuous_scale = color_continuous_scale, zoom = 10,
                        hover_data = ['nbrhd'])
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

### 10 parcels (blue), 100 violations (yellow)

In [ ]:
np.random.seed(0)

toPlot = pd.concat([parcels.sample(10)[['lat', 'lng']].assign(type = 'parcel'), viols.sample(100)[['lat', 'lng']].assign(type = 'viol')], axis = 0)

cmap = {
    'viol': 'rgba(253,231,37,0.8)',
    'parcel': '#4169E1'
}

fig = px.scatter_mapbox(toPlot, lat = 'lat', lon = 'lng', color = 'type', color_discrete_map = cmap, zoom = 10)
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

### Construct X-meter radius

In [ ]:
def construct_perimeter(parcels, r):
    x_vals = [i for i in range(0, r + 1, 1)]
    y_vals = []

    for x in x_vals:
        y = np.sqrt( (r ** 2) - (x ** 2) )
        y_vals.append(y)
        pass
    
    conversionVector = np.array([85500, 111111.111])
    coords1 = np.array([[i, j] for i, j in zip(x_vals, y_vals)]) / conversionVector
    coords2 = coords1 * np.array([-1, 1])
    coords3 = coords1 * np.array([-1, -1])
    coords4 = coords1 * np.array([1, -1])
    
    polyX_vals = []

    for index, row in parcels.iterrows():
        origin = np.array([row['lng'], row['lat']])
        coordsX_1 = origin + coords1
        coordsX_2 = origin + coords2
        coordsX_3 = origin + coords3
        coordsX_4 = origin + coords4
        coordsX = np.concatenate((coordsX_1, coordsX_2, coordsX_3, coordsX_4))
        polyX = Polygon(coordsX)
        polyX_vals.append(polyX)
        pass
    
    return polyX_vals

### 3 parcels and 750m radius (blue), 100 violations (purple)

In [ ]:
np.random.seed(0)
three_parcels = parcels.loc[parcels['blight_sqrt'] == 6].sample(3)
three_parcels['radius_750'] = construct_perimeter(three_parcels, 750)
perimeter_coords = (
    pd.DataFrame([three_parcels['radius_750'].values[i].exterior.xy for i in range(three_parcels.shape[0])]).rename(columns = {0: 'lng', 1: 'lat'})
    .apply(pd.Series.explode).reset_index(drop = True)
    .assign(type = 'parcel')
)
toPlot = pd.concat([
    three_parcels[['lat', 'lng']].assign(type = 'parcel'), perimeter_coords, viols.sample(100)[['lat', 'lng']].assign(type = 'viol')
], axis = 0)

cmap = {
    'viol': 'rgba(68,1,84,0.8)',
    'parcel': '#4169E1'
}

fig = px.scatter_mapbox(toPlot, lat = 'lat', lon = 'lng', color = 'type', color_discrete_map = cmap, zoom = 10)
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

### One parcel, every radius bin

In [ ]:
np.random.seed(0)
one_parcel = parcels.loc[parcels['blight_sqrt'] == 6].sample(1)
bins = np.arange(50, 800, 50)
perimeters = pd.DataFrame()
for bin in bins:
    one_parcel[f'radius_{bin}'] = construct_perimeter(one_parcel, bin)
    perimeter_coords = (
        pd.DataFrame([one_parcel[f'radius_{bin}'].values[i].exterior.xy for i in range(one_parcel.shape[0])]).rename(columns = {0: 'lng', 1: 'lat'})
        .apply(pd.Series.explode).reset_index(drop = True)
        .assign(radius = bin)
    )
    perimeters = pd.concat([perimeters, perimeter_coords], axis = 0)

toPlot = pd.concat([perimeters, one_parcel[['lat', 'lng']].assign(radius = 0)], axis = 0)

cmap = [
    (0.0, '#4169E1'),  # start
    (1.0, '#87CEFA')   # end
]

fig = px.scatter_mapbox(toPlot, lat = 'lat', lon = 'lng', color = 'radius', color_continuous_scale = cmap, zoom = 10)
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

### Continuous property values

In [ ]:
toPlot = (
    parcels
    .assign(
        values_smoothed = np.sqrt(parcels['values_smoothed'])
    )
    .groupby(['parcelID', 'lat', 'lng'])['values_smoothed'].mean().reset_index()
)
toPlot.head(1)

In [ ]:
np.random.seed(0)
km = KMeans(n_clusters = 7)
toPlot['cluster'] = km.fit_predict(pd.DataFrame(toPlot['values_smoothed']))
toPlot.groupby('cluster')['values_smoothed'].mean()

In [ ]:
toPlot['cluster'] = toPlot['cluster'].map({
    2: 1,
    6: 2,
    0: 3,
    5: 4,
    3: 5,
    1: 6,
    4: 7
})

In [ ]:
fig = px.scatter_mapbox(toPlot, lat = 'lat', lon = 'lng', color = 'cluster', color_continuous_scale = color_continuous_scale, zoom = 10)
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()

In [ ]:
np.random.seed(0)
oneParcel = parcels.loc[parcels['blight'] == '6'].sample(2)

In [ ]:
oneParcel['radius_750'] = construct_perimeter(oneParcel, 750)
perimeterCoords = (
    pd.DataFrame([oneParcel['radius_750'].values[i].exterior.xy for i in range(2)]).rename(columns = {0: 'lng', 1: 'lat'})
    .apply(pd.Series.explode).reset_index(drop = True)
) 
toPlot = pd.concat([oneParcel[['lng', 'lat']], perimeterCoords], axis = 0)

In [ ]:
fig = px.scatter_mapbox(toPlot, lat = 'lat', lon = 'lng', zoom = 10)
fig.update_layout(mapbox_style = 'open-street-map')
fig.show()

### Overlay demos on top of blight backdrop

In [ ]:
demos = pd.read_csv(clean + 'clean_demos.csv').query('status == "Completed"').reset_index(drop = True)

In [ ]:
toPlot = (
    parcels
    .merge(right = demos[['parcelID']].assign(demo = 1), on = 'parcelID', how = 'left')
    .assign(
        blight_sqrt = lambda x: np.where(x['demo'] == 1, 'Demo', x['blight_sqrt'])
    )
    [['lat', 'lng', 'blight_sqrt', 'nbrhd']].rename(columns = {'blight_sqrt': 'Blight', 'nbrhd': 'Neighborhood'})
)

color_discrete_map = {
    '1': '#440154',
    '2': '#443983',
    '3': '#31688e',
    '4': '#21918c',
    '5': '#35b779',
    '6': '#90d743',
    '7': '#fde725',
    'Demo': '#FFFFFF'
}

fig = px.scatter_mapbox(toPlot, lat = 'lat', lon = 'lng', color = 'Blight', color_discrete_map = color_discrete_map, zoom = 10,
                        category_orders = {'Blight': ['1', '2', '3', '4', '5', '6', '7', 'Demo']}, hover_data = ['Neighborhood'])
fig.update_layout(mapbox_style = 'carto-positron')
fig.show()